# MODEL

In [29]:
# Gerekli kütüphanler

import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error , r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import model_selection
import matplotlib.pyplot as plt
from sklearn.linear_model import RidgeCV

In [8]:
# Veri Seti (Beyzbol ile ilgili terimlerden oluşuyor.)

df = pd.read_csv("C:\python_py\Makine Öğrenmesi-2\Hitters.csv")
df = df.dropna()
dms = pd.get_dummies(df[["League" , "Division" , "NewLeague"]])     # kategorik değişkenleri dummy değişkene çevirdik.
y = df["Salary"]

# Özetle bu iki satırda yaptığımız işlem:
# kategorik değişkenleri dummy değişkenlere çevirip dummy değişkenleri veri setinde tutarak diğer değişkenleri ve bağımlı değişkeni bağımsızların dışında bıraktık.
x__ = df.drop(["Salary" , "League" , "Division" , "NewLeague"] , axis=1).astype("float64")     # Dummy oluşturduğumuz için kategorik değişkenlerin kendilerini de attık. 
x = pd.concat([x__ , dms[["League_N" , "Division_W" , "NewLeague_N"]]] , axis=1)       # bağımsız değişkenlerimizin nihai halini oluşturmuş olduk (oluşturduğumuz dummy'ler + üstteki bağımsız değişkenler)


x_train , x_test , y_train , y_test = train_test_split(x , y , test_size=0.25 , random_state=42)

In [9]:
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N
5,594,169,4,74,51,35,11,4408,1133,19,501,336,194,A,W,282,421,25,750.0,A


In [11]:
df.shape

(263, 20)

- Ridge Regresyondaki Alfa (= formülümüzdeki lambda λ parametresi) Nedir?


Alfa (= formülümüzdeki lambda λ parametresi), Ridge regresyonunun düzenlileştirme etkisini kontrol eden bir parametredir. Bu parametre, regresyon katsayılarının ne kadar büyük olmasına izin verileceğini belirler. Daha büyük bir alfa değeri, katsayıların daha küçük ve sıkışık olmasına yol açar, bu da modelin aşırı uyum riskini azaltabilir, ancak aynı zamanda modelin esnekliğini kısıtlayabilir. Daha küçük bir alfa değeri, katsayıların daha büyük olmasına izin verir ve modelin verilere daha yakın oturmasına neden olabilir, ancak aşırı uyum riski artabilir.

In [19]:
ridge_model = Ridge(alpha=0.1).fit(x_train , y_train)    # fit etmeyi bu şekilde de gerçekleştirebiliriz. -> model kurdum demek bu kadar aslında

In [21]:
ridge_model.coef_   # b1 , b2 , b3...



array([ -1.77435737,   8.80240528,   7.29595605,  -3.33257639,
        -2.08316481,   5.42531283,   7.58514945,  -0.13752764,
        -0.20779701,  -0.60361067,   1.7927957 ,   0.72866408,
        -0.68710375,   0.26153564,   0.26888652,  -0.52674278,
       112.14640272, -99.80997876, -48.07152768])

In [22]:
ridge_model.intercept_  # b0

-4.5786269057206255

# TAHMİN

In [24]:
y_pred = ridge_model.predict(x_train)
y_pred  # tahmin edilen değerler

array([ 377.32722972,  802.33593374,  495.66497455,  112.55723373,
        426.36377192, 1004.17111652,  154.60488574,  362.44112621,
        485.16325501,  918.52163204, 1621.10775334,  818.04761824,
        920.88184597,  739.20423491,  645.71641635,  706.93566474,
       1139.9491843 ,   40.91367448,  213.27383013,  406.89150389,
        709.89358202, 1085.20352009,  474.18037697,  392.47016564,
         13.90916672,  757.41269763,  561.32351629,  186.61572852,
        339.30739236,  319.84763625,  203.51136096,  500.63531441,
        949.39753214,  299.61528061,  492.72700962,  425.29372691,
        421.50833755,  676.37246687,  292.01621128,  204.7820655 ,
        214.70918789,  322.02090488,  202.20359566,  133.93043147,
        226.4089696 , 1089.86826738,  331.04046671,  530.27858352,
        343.24418499,  322.75794204,  539.36798843,  442.83336413,
        331.49041516,  450.80873849,  655.78728069,  282.26455005,
        452.81925756,  713.14983579,  270.24840046, 1530.11024

In [25]:
y_train # gerçek değerler

183     125.000
229    1940.000
286     940.000
102     375.000
153     100.000
         ...   
24      776.667
236     750.000
93      145.000
137     430.000
133      87.500
Name: Salary, Length: 197, dtype: float64

In [27]:
# Eğitim setimizin hata(RMSE) oranına bakalım -> Genellikle model tuning bölümünde modelleri doğrulamak (en iyi parametreleri seçmek) için uygulayacağız

RMSE = np.sqrt(mean_squared_error(y_train , y_pred))
RMSE

289.3294761170452

In [31]:
# Çapraz doğrulanmış hata oranımız -> Eğitim seti üzerine uygulayacağız model tuningde

np.mean(-cross_val_score(ridge_model , x_train , y_train , cv=10 , scoring="neg_mean_squared_error"))

123744.6504693825

In [32]:
# Çapraz doğrulanmış hata kareler ortalamasının karekök değeri -> DOĞRULANMIŞ HATA ORANIMIZ

np.sqrt(np.mean(-cross_val_score(ridge_model , x_train , y_train , cv=10 , scoring="neg_mean_squared_error")))

# RMSE ve bu hata oranını karşılaştır. Bu hata oranının daha düşük olaması daha iyi anlama gelmez. Bu son bulduğumuz hata oranı daha doğru bir hata oranı. Bu hata oranına göre hareket edersek daha doğru

351.77357841285135

In [34]:
# Test setimizin hata oranı

y_pred = ridge_model.predict(x_test)    # x'in test setinin bağımsız değişkenlerini kullanarak y'yi tahmin ettik

RMSE = np.sqrt(mean_squared_error(y_test , y_pred))     # daha sonra bu tahmin ettiğimiz y ile gerçek y yi karşılaştırdık
RMSE

357.0239252607652

# MODEL TUNİNG

    Neden Model Tuning?

Bize en az hata oranını verecek olan alfa (lambda) değerini bulmak için 

In [35]:
ridge_model = Ridge().fit(x_train , y_train)
y_pred = ridge_model.predict(x_test)
np.sqrt(mean_squared_error(y_test , y_pred))

356.8082905730236

In [37]:
lambdalar1 = np.random.randint(0,1000,100)

array([110, 920, 922, 695, 819,  68, 561, 715, 486, 679, 549, 494, 907,
       384, 809, 215, 332, 139, 716, 612, 172, 224, 462, 683, 945, 164,
       580, 808, 244, 959, 104, 781, 771, 400, 305, 395, 549, 824,  33,
       463,  60, 953, 402, 194, 953,  13, 272, 936, 383, 275, 161, 581,
       538, 253, 224, 985, 867, 118, 605, 815, 910, 288, 573, 967, 967,
       467, 649, 244, 606, 892,  52, 696, 315, 794, 732, 827, 283, 982,
       142, 201, 486, 351, 346, 213, 633, 838, 519, 862,   8, 705, 269,
       118, 448, 783, 189,  12, 331, 551, 250, 237])

In [45]:
lambdalar2 = 10**np.linspace(10,-2,100)*0.5

- Farklı lambda değerlerine karşılık olarak oluşabilecek farklı katsayılara ilişkin hatalarımızı incelemeye çalışacağız -> optimum alfa (lambda) değerini bulacağız

In [46]:
ridgecv = RidgeCV(alphas=lambdalar2 , scoring="neg_mean_squared_error" , cv=10).fit(x_train , y_train)     # Modeli kurduk

RidgeCV(alphas=array([5.00000000e+09, 3.78231664e+09, 2.86118383e+09, 2.16438064e+09,
       1.63727458e+09, 1.23853818e+09, 9.36908711e+08, 7.08737081e+08,
       5.36133611e+08, 4.05565415e+08, 3.06795364e+08, 2.32079442e+08,
       1.75559587e+08, 1.32804389e+08, 1.00461650e+08, 7.59955541e+07,
       5.74878498e+07, 4.34874501e+07, 3.28966612e+07, 2.48851178e+07,
       1.88246790e+07, 1.42401793e+0...
       3.06795364e+00, 2.32079442e+00, 1.75559587e+00, 1.32804389e+00,
       1.00461650e+00, 7.59955541e-01, 5.74878498e-01, 4.34874501e-01,
       3.28966612e-01, 2.48851178e-01, 1.88246790e-01, 1.42401793e-01,
       1.07721735e-01, 8.14875417e-02, 6.16423370e-02, 4.66301673e-02,
       3.52740116e-02, 2.66834962e-02, 2.01850863e-02, 1.52692775e-02,
       1.15506485e-02, 8.73764200e-03, 6.60970574e-03, 5.00000000e-03]),
        cv=10, scoring='neg_mean_squared_error')

In [47]:
ridgecv.alpha_  # optimum yani olması gereken alfa(lambda) değeri

1526927.7544167063

In [51]:
# Final Modeli

ridge_tuned = Ridge(alpha=ridgecv.alpha_).fit(x_train , y_train)

y_pred = ridge_model.predict(x_test)
np.sqrt(mean_squared_error(y_test , y_pred))

356.8082905730236